In [1]:
from rcsbapi.data.query import Query
from rcsbapi.data.query import SCHEMA
from rcsbsearchapi.search import TextQuery
from rcsbsearchapi import rcsb_attributes as attrs
from pprint import pprint

## RCSB PDB Data API: Search and Data API Workflow Demo

This quick-start notebook will walk through the basics of making queries in this package using a simple example. For more in-depth documentation reference the [README](https://github.com/rcsb/py-rcsb-api/blob/master/README.md).

\
install the package: 

```pip install rcsbdataapi```

\
In this demo, we are interested in ligand-bound COVID-19 virus structures. In this demo, we will:
 1. Python Search API package: construct a query to fetch COVID-19 viruses with ligands bound
 2. Python Data API package: find information about each ligand (PDB ID, Formula, associated publications)

### Python Search API: Find COVID-19 structures with ligand bound
To learn more about using the Search API Python package, read the documentation [here](https://rcsbsearchapi.readthedocs.io/en/latest/)

We'll start by constructing a query where the source organism is "COVID-19 virus" and where there is a nonpolymer_enitity that is the subject of investigation in the structure, meaning that the research associated on the structure focuses on the bound non polymer entity ligand.

- Uses python comparison operators for basic attributes (==, <, <=, etc)
- Combine using set operators (&, |, ~, etc)
- Execute queries as functions


In [4]:
# Create each subquery
q1 = attrs.rcsb_entity_source_organism.taxonomy_lineage.name == "COVID-19 virus"
q2 = attrs.rcsb_nonpolymer_entity_annotation.type == "SUBJECT_OF_INVESTIGATION"

# combined using bitwise operators (&, |, ~, etc)
query = q1 & q2

# Call the query to execute it
for result in query():
    print(result)


5R7Y
5R7Z
5R80
5R81
5R82
5R83
5R84
5RE4
5RE5
5RE6
5RE7
5RE8
5RE9
5REA
5REB
5REC
5RED
5REE
5REF
5REG
5REH
5REI
5REJ
5REK
5REL
5REM
5REN
5REO
5REP
5RER
5RES
5RET
5REU
5REV
5REW
5REX
5REY
5REZ
5RF0
5RF1
5RF2
5RF3
5RF4
5RF5
5RF6
5RF7
5RF8
5RF9
5RFA
5RFB
5RFC
5RFD
5RFE
5RFF
5RFG
5RFH
5RFI
5RFJ
5RFK
5RFL
5RFM
5RFN
5RFO
5RFP
5RFQ
5RFR
5RFS
5RFT
5RFU
5RFV
5RFW
5RFX
5RFY
5RFZ
5RG0
5RG1
5RG2
5RG3
5RGG
5RGH
5RGI
5RGJ
5RGK
5RGL
5RGM
5RGN
5RGO
5RGP
5RGQ
5RGR
5RGS
5RGT
5RGU
5RGV
5RGW
5RGX
5RGY
5RGZ
5RH0
5RH1
5RH2
5RH3
5RH4
5RH5
5RH6
5RH7
5RH8
5RH9
5RHA
5RHF
5RL1
5RL2
5RL3
5RL4
5RL5
5RL6
5RL7
5RL8
5RL9
5RLB
5RLC
5RLD
5RLE
5RLF
5RLG
5RLH
5RLI
5RLJ
5RLK
5RLL
5RLM
5RLN
5RLO
5RLP
5RLQ
5RLR
5RLS
5RLT
5RLU
5RLV
5RLW
5RLY
5RLZ
5RM0
5RM1
5RM2
5RM3
5RM4
5RM5
5RM6
5RM7
5RM8
5RM9
5RMA
5RMB
5RMC
5RMD
5RME
5RMF
5RMG
5RMH
5RMI
5RMJ
5RMK
5RML
5RMM
5RS7
5RS8
5RS9
5RSB
5RSC
5RSD
5RSE
5RSF
5RSG
5RSH
5RSI
5RSJ
5RSK
5RSL
5RSM
5RSN
5RSO
5RSP
5RSQ
5RSR
5RSS
5RST
5RSU
5RSV
5RSW
5RSX
5RSY
5RSZ
5RT0
5RT1
5RT2
5RT3
5RT4
5RT5


: 